# FastAPI Gemini Chatbot Demo

This notebook demonstrates the FastAPI chatbot with Gemini LLM integration, featuring:
- Model routing based on content complexity
- Google Search grounding with citations
- Code execution capabilities
- Conversation management
- CRUD endpoints for chat history

## Setup and Dependencies

Install required packages and import libraries for testing the API.

In [3]:
# Install required packages
!pip install requests python-dotenv fastapi uvicorn google-generativeai

## Start the FastAPI Server

Before running the tests, you need to start the FastAPI server. You can do this in several ways:

In [4]:
# Option 1: Start server in background (recommended)
import subprocess
import time
import os
import signal

# Check if server is already running
def check_server():
    try:
        response = requests.get(f"{BASE_URL}/docs", timeout=5)
        return response.status_code == 200
    except:
        return False

# Start the FastAPI server in background
def start_server():
    if check_server():
        print("✅ Server is already running!")
        return None
    
    print("🚀 Starting FastAPI server...")
    # Change to the directory containing main.py
    server_dir = "/Users/wahyu/Code/H8/data-scientist/final-project"
    
    # Start server as background process
    process = subprocess.Popen(
        ["python", "-m", "uvicorn", "main:app", "--reload", "--port", "8001"],
        cwd=server_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    # Wait a bit for server to start
    time.sleep(3)
    
    if check_server():
        print("✅ Server started successfully!")
        return process
    else:
        print("❌ Failed to start server. Check if main.py exists and GEMINI_API_KEY is set.")
        return None

# Start the server
server_process = start_server()

🚀 Starting FastAPI server...
❌ Failed to start server. Check if main.py exists and GEMINI_API_KEY is set.
❌ Failed to start server. Check if main.py exists and GEMINI_API_KEY is set.


**Alternative: Manual Server Start**

If the above doesn't work, open a terminal and run:
```bash
cd /Users/wahyu/Code/H8/data-scientist/final-project
python -m uvicorn main:app --reload --port 8001
```

Make sure you have:
1. Set the `GEMINI_API_KEY` environment variable
2. Installed all dependencies: `pip install fastapi uvicorn google-generativeai python-dotenv`

In [5]:
# Check if server is running and accessible
print("🔍 Checking server status...")

if check_server():
    print("✅ Server is running and accessible!")
    print(f"📊 API Documentation: {BASE_URL}/docs")
    print(f"📋 OpenAPI Schema: {BASE_URL}/openapi.json")
else:
    print("❌ Server is not accessible. Please start the server first.")
    print("\nTo start manually:")
    print("1. Open terminal")
    print("2. cd /Users/wahyu/Code/H8/data-scientist/final-project")
    print("3. python -m uvicorn main:app --reload --port 8001")

🔍 Checking server status...
❌ Server is not accessible. Please start the server first.

To start manually:
1. Open terminal
2. cd /Users/wahyu/Code/H8/data-scientist/final-project
3. python -m uvicorn main:app --reload --port 8001


In [6]:
import requests
import json
import time
from datetime import datetime

# API base URL (assuming server runs on localhost:8001)
BASE_URL = "http://localhost:8001"

# Test user and chat IDs
TEST_USER_ID = "test_user_123"
TEST_CHAT_ID = "chat_" + str(int(time.time()))

## Helper Functions

Define functions to interact with the chatbot API.

In [7]:
def send_message(user_id, chat_id, message_content, role="user"):
    """Send a message to the chatbot API."""
    url = f"{BASE_URL}/chat"
    payload = {
        "user_id": user_id,
        "chat_id": chat_id,
        "message": {
            "role": role,
            "content": message_content
        }
    }
    
    try:
        response = requests.post(url, json=payload, timeout=30)
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def get_chat_history(user_id, chat_id):
    """Get chat history for a specific conversation."""
    url = f"{BASE_URL}/user/{user_id}/chat/{chat_id}"
    try:
        response = requests.get(url)
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def get_all_chats(user_id):
    """Get all chat IDs and titles for a user."""
    url = f"{BASE_URL}/user/{user_id}/chats"
    try:
        response = requests.get(url)
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def delete_chat(user_id, chat_id):
    """Delete a specific chat conversation."""
    url = f"{BASE_URL}/user/{user_id}/chat/{chat_id}"
    try:
        response = requests.delete(url)
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def print_response(response, show_citations=True):
    """Pretty print API response."""
    if "error" in response:
        print(f"❌ Error: {response['error']}")
        return
    
    if "response" in response:
        print(f"🤖 Assistant ({response.get('model', 'unknown')}):\n{response['response']['content']}")
        
        if show_citations and response.get('citation'):
            print(f"\n📚 {response['citation']}")
        
        if response.get('title'):
            print(f"\n📝 Chat Title: {response['title']}")
    else:
        print(json.dumps(response, indent=2))

## Test 1: Basic Conversation

Start with simple questions to test basic functionality and model routing.

In [8]:
# Simple math question (should use flash-lite model)
print("=== Simple Math Question ===")
response = send_message(TEST_USER_ID, TEST_CHAT_ID, "What is 2 + 2?")
print_response(response)

=== Simple Math Question ===
🤖 Assistant (gemini-2.5-flash-lite):
print(2 + 2)Output: 4
4

📝 Chat Title: Simple Math Question
🤖 Assistant (gemini-2.5-flash-lite):
print(2 + 2)Output: 4
4

📝 Chat Title: Simple Math Question


In [9]:
# Medium complexity question (should use flash model)
print("=== Medium Complexity Question ===")
response = send_message(TEST_USER_ID, TEST_CHAT_ID, "Jelaskan apa itu machine learning? dan jelaskan neural network?")
print_response(response)

=== Medium Complexity Question ===
🤖 Assistant (gemini-2.5-flash):
Machine learning (Pembelajaran Mesin) adalah cabang dari kecerdasan buatan (AI) yang memungkinkan sistem untuk belajar dari data, mengidentifikasi pola, dan membuat keputusan dengan intervensi manusia yang minimal. Alih-alih diprogram secara eksplisit untuk setiap tugas, sistem machine learning menggunakan algoritma yang memungkinkan mereka untuk "belajar" dari sejumlah besar data. Setelah belajar, mereka dapat menerapkan pengetahuan ini untuk membuat prediksi atau keputusan tentang data baru yang belum pernah mereka lihat sebelumnya.

Neural Network (Jaringan Saraf Tiruan) adalah sebuah model machine learning yang terinspirasi dari struktur dan fungsi otak manusia. Jaringan saraf tiruan terdiri dari lapisan-lapisan "node" atau "neuron" yang saling terhubung. Setiap neuron menerima input, memprosesnya, dan mengirimkan output ke neuron lain di lapisan berikutnya.

Berikut adalah komponen utamanya:
*   **Lapisan Input**: 

In [10]:
# Hard complexity question (should use pro model)
print("=== Hard Complexity Question ===")
response = send_message(TEST_USER_ID, TEST_CHAT_ID, "Explain the mathematical proof of the derivative of x^n using first principles and provide the algorithm for implementing it.")
print_response(response)

=== Hard Complexity Question ===
🤖 Assistant (gemini-2.5-flash):
The derivative of a function $f(x)$ using first principles (also known as the limit definition of the derivative) is given by:

$f'(x) = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h}$

For the function $f(x) = x^n$, we need to find:

$\frac{d}{dx}(x^n) = \lim_{h \to 0} \frac{(x + h)^n - x^n}{h}$

**Mathematical Proof:**

1.  **Expand $(x + h)^n$ using the Binomial Theorem:**
The Binomial Theorem states that $(a + b)^n = \sum_{k=0}^{n} \binom{n}{k} a^{n-k} b^k$.
Applying this to $(x + h)^n$ with $a=x$ and $b=h$:
$(x + h)^n = \binom{n}{0} x^n h^0 + \binom{n}{1} x^{n-1} h^1 + \binom{n}{2} x^{n-2} h^2 + \dots + \binom{n}{n} x^0 h^n$
$(x + h)^n = 1 \cdot x^n \cdot 1 + n \cdot x^{n-1} \cdot h + \frac{n(n-1)}{2!} x^{n-2} h^2 + \dots + h^n$
$(x + h)^n = x^n + n x^{n-1} h + \frac{n(n-1)}{2} x^{n-2} h^2 + \dots + h^n$

2.  **Substitute this expansion back into the limit definition:**
$\frac{(x + h)^n - x^n}{h} = \frac{(x^n + n x^{n-1} h

## Test 2: Code Execution

Test the code execution capabilities with programming questions.

In [11]:
# Code generation and execution
print("=== Code Execution Test ===")
response = send_message(TEST_USER_ID, TEST_CHAT_ID, "Write a Python function to calculate the factorial of a number and show the result for factorial of 5.")
print_response(response)

=== Code Execution Test ===
🤖 Assistant (gemini-2.5-flash-lite):
def factorial(n):
if n == 0:
return 1
else:
return n * factorial(n-1)

print(factorial(5))Output: 120
The factorial of 5 is 120.

📝 Chat Title: Simple Math Question
🤖 Assistant (gemini-2.5-flash-lite):
def factorial(n):
if n == 0:
return 1
else:
return n * factorial(n-1)

print(factorial(5))Output: 120
The factorial of 5 is 120.

📝 Chat Title: Simple Math Question


In [12]:
# Complex math calculation
print("=== Complex Math Calculation ===")
response = send_message(TEST_USER_ID, TEST_CHAT_ID, "Calculate the first 10 prime numbers and show their sum.")
print_response(response)

=== Complex Math Calculation ===
🤖 Assistant (gemini-2.5-flash-lite):
def is_prime(num):
if num < 2:
return False
for i in range(2, int(num**0.5) + 1):
if num % i == 0:
return False
return True

prime_numbers = []
num = 2
while len(prime_numbers) < 10:
if is_prime(num):
prime_numbers.append(num)
num += 1

sum_of_primes = sum(prime_numbers)

print("The first 10 prime numbers are:", prime_numbers)
print("Their sum is:", sum_of_primes)Output: The first 10 prime numbers are: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Their sum is: 129
The first 10 prime numbers are:.
Their sum is 129.

📝 Chat Title: Simple Math Question
🤖 Assistant (gemini-2.5-flash-lite):
def is_prime(num):
if num < 2:
return False
for i in range(2, int(num**0.5) + 1):
if num % i == 0:
return False
return True

prime_numbers = []
num = 2
while len(prime_numbers) < 10:
if is_prime(num):
prime_numbers.append(num)
num += 1

sum_of_primes = sum(prime_numbers)

print("The first 10 prime numbers are:", prime_numbers)
print("Their sum

## Test 3: Google Search Grounding

Test the Google Search integration for getting current information with citations.

In [13]:
# Question that should trigger Google Search
print("=== Google Search Test ===")
response = send_message(TEST_USER_ID, TEST_CHAT_ID, "What are the latest news about artificial intelligence in 2025?")
print_response(response)

=== Google Search Test ===
🤖 Assistant (gemini-2.5-flash-lite):
I cannot provide real-time news updates for the future. My knowledge base is current up to my last update, and I don't have access to future events or news.

However, I can tell you about general trends and areas of focus that are likely to continue in AI development into 2025:

*   **Generative AI Advancements:** Expect continued rapid progress in models that can generate text, images, audio, and even video. This includes more sophisticated and controllable creative tools, as well as applications in areas like drug discovery and material science.
*   **Responsible AI and Ethics:** As AI becomes more pervasive, there will be a strong emphasis on developing and deploying AI systems ethically and responsibly. This includes addressing issues of bias, transparency, accountability, and privacy.
*   **AI in Healthcare:** AI will likely play an even larger role in diagnostics, personalized medicine, drug development, and administ

In [14]:
# Recipe search (Indonesian content)
print("=== Recipe Search Test ===")
response = send_message(TEST_USER_ID, TEST_CHAT_ID, "Cari resep nasi goreng yang enak dan mudah dibuat")
print_response(response)

=== Recipe Search Test ===
🤖 Assistant (gemini-2.5-flash-lite):
Tentu, ini resep nasi goreng yang enak dan mudah dibuat:

**Nasi Goreng Kampung Sederhana**

Resep ini sangat fleksibel, Anda bisa menambahkan bahan lain sesuai selera.

**Bahan-bahan:**

*   2 piring nasi putih dingin (nasi kemarin lebih baik agar tidak lembek)
*   2 siung bawang putih, cincang halus
*   3 siung bawang merah, iris tipis
*   2 buah cabai merah keriting (sesuai selera), iris serong
*   1 butir telur ayam
*   50 gram teri medan (opsional, rendam air panas sebentar lalu tiriskan)
*   1 sdm kecap manis
*   1 sdt saus tiram
*   1/2 sdt garam (sesuaikan selera)
*   1/4 sdt merica bubuk
*   1 batang daun bawang, iris tipis (untuk taburan)
*   Minyak goreng secukupnya

**Bahan Pelengkap (opsional):**

*   Kerupuk
*   Acar timun
*   Irisan tomat dan timun
*   Telur mata sapi

**Cara Membuat:**

1.  **Siapkan Wajan:** Panaskan sedikit minyak goreng dalam wajan dengan api sedang.
2.  **Tumis Bumbu:** Masukkan bawang 

## Test 4: Conversation Management

Test conversation history and chat management features.

In [15]:
# Get chat history
print("=== Chat History ===")
history = get_chat_history(TEST_USER_ID, TEST_CHAT_ID)
print(json.dumps(history, indent=2))

=== Chat History ===
{
  "title": "Simple Math Question",
  "messages": [
    {
      "role": "user",
      "content": "What is 2 + 2?"
    },
    {
      "role": "assistant",
      "content": "print(2 + 2)Output: 4\n4"
    },
    {
      "role": "user",
      "content": "Jelaskan apa itu machine learning? dan jelaskan neural network?"
    },
    {
      "role": "assistant",
      "content": "Machine learning (Pembelajaran Mesin) adalah cabang dari kecerdasan buatan (AI) yang memungkinkan sistem untuk belajar dari data, mengidentifikasi pola, dan membuat keputusan dengan intervensi manusia yang minimal. Alih-alih diprogram secara eksplisit untuk setiap tugas, sistem machine learning menggunakan algoritma yang memungkinkan mereka untuk \"belajar\" dari sejumlah besar data. Setelah belajar, mereka dapat menerapkan pengetahuan ini untuk membuat prediksi atau keputusan tentang data baru yang belum pernah mereka lihat sebelumnya.\n\nNeural Network (Jaringan Saraf Tiruan) adalah sebuah model

In [16]:
# Get all chats for the user
print("=== All User Chats ===")
all_chats = get_all_chats(TEST_USER_ID)
print(json.dumps(all_chats, indent=2))

=== All User Chats ===
{
  "chats": [
    {
      "chat_id": "chat_1755697807",
      "title": "Simple Math Question"
    }
  ]
}


## Test 5: Multiple Conversations

Create multiple chat sessions to test conversation isolation.

In [17]:
# Create a new chat session
NEW_CHAT_ID = "chat_" + str(int(time.time()) + 1)

print("=== New Chat Session ===")
response = send_message(TEST_USER_ID, NEW_CHAT_ID, "Hello! Can you help me with data science?")
print_response(response)

print("\n=== Follow-up in New Chat ===")
response = send_message(TEST_USER_ID, NEW_CHAT_ID, "What are the main steps in a machine learning pipeline?")
print_response(response)

=== New Chat Session ===
🤖 Assistant (gemini-2.5-flash-lite):
Hello! I'd be happy to help you with data science. What specifically are you working on or curious about? For example, you could ask me about:

*   **Specific algorithms:** "Can you explain logistic regression?" or "How does a random forest work?"
*   **Libraries and tools:** "What are the advantages of using Pandas?" or "How do I install scikit-learn?"
*   **Concepts:** "What is overfitting?" or "Can you explain the bias-variance tradeoff?"
*   **Tasks:** "How do I perform data cleaning?" or "What are some ways to visualize data?"
*   **Coding help:** "Can you show me a Python snippet for calculating the mean of a list?"

The more specific you are, the better I can assist you!

📝 Chat Title: Data Science Assistance

=== Follow-up in New Chat ===
🤖 Assistant (gemini-2.5-flash-lite):
Hello! I'd be happy to help you with data science. What specifically are you working on or curious about? For example, you could ask me about:



In [18]:
# Check all chats again
print("=== Updated Chat List ===")
all_chats = get_all_chats(TEST_USER_ID)
print(json.dumps(all_chats, indent=2))

=== Updated Chat List ===
{
  "chats": [
    {
      "chat_id": "chat_1755697807",
      "title": "Simple Math Question"
    },
    {
      "chat_id": "chat_1755697866",
      "title": "Data Science Assistance"
    }
  ]
}


## Test 6: Timeout Testing

Test the 25-second timeout functionality with a complex request.

In [19]:
# Complex request that might take time
print("=== Timeout Test ===")
start_time = time.time()
response = send_message(TEST_USER_ID, NEW_CHAT_ID, 
    "Generate a comprehensive analysis of the latest machine learning research papers, "
    "including their methodologies, results, and implications for the field. "
    "Also search for the most recent developments and provide detailed citations.")
end_time = time.time()

print(f"Request took {end_time - start_time:.2f} seconds")
print_response(response)

=== Timeout Test ===
Request took 3.78 seconds
🤖 Assistant (gemini-2.5-flash-lite):
I can help you find information about recent machine learning research papers. However, generating a *comprehensive analysis* of the *latest* papers, including their methodologies, results, and implications, would require a deep dive into numerous highly technical documents. This is something that would typically be done by a human researcher over a significant period.

What I *can* do is search for highly cited or influential recent papers in specific areas of machine learning or on particular topics you're interested in. This would give you a starting point for your own analysis.

To help me narrow down the search, could you tell me:

1.  **What specific areas of machine learning are you most interested in?** (e.g., Natural Language Processing, Computer Vision, Reinforcement Learning, Explainable AI, etc.)
2.  **Are there any particular keywords or topics you'd like to focus on?** (e.g., "large langua

## Test 7: Error Handling

Test various error scenarios and edge cases.

In [20]:
# Test with empty message
print("=== Empty Message Test ===")
response = send_message(TEST_USER_ID, NEW_CHAT_ID, "")
print_response(response)

# Test with very long message
print("\n=== Very Long Message Test ===")
long_message = "This is a very long message. " * 100
response = send_message(TEST_USER_ID, NEW_CHAT_ID, long_message)
print_response(response)

=== Empty Message Test ===
🤖 Assistant (gemini-2.5-flash-lite):
I'm ready when you are! Just let me know what area of machine learning you'd like to explore, or if you have any specific questions.

📝 Chat Title: Data Science Assistance

=== Very Long Message Test ===
🤖 Assistant (gemini-2.5-flash-lite):
I'm ready when you are! Just let me know what area of machine learning you'd like to explore, or if you have any specific questions.

📝 Chat Title: Data Science Assistance

=== Very Long Message Test ===
🤖 Assistant (gemini-2.5-flash-lite):
It seems like you've pasted a very long message, repeating the phrase "This is a very long message."

Is there something specific you'd like to discuss or a question you have about data science or machine learning? I'm here to help!

📝 Chat Title: Data Science Assistance
🤖 Assistant (gemini-2.5-flash-lite):
It seems like you've pasted a very long message, repeating the phrase "This is a very long message."

Is there something specific you'd like to d

## Test 8: Chat Deletion

Test the chat deletion functionality.

In [21]:
# Delete one of the test chats
print("=== Delete Chat Test ===")
delete_response = delete_chat(TEST_USER_ID, NEW_CHAT_ID)
print(json.dumps(delete_response, indent=2))

# Verify deletion
print("\n=== Chat List After Deletion ===")
all_chats = get_all_chats(TEST_USER_ID)
print(json.dumps(all_chats, indent=2))

=== Delete Chat Test ===
{
  "message": "Chat chat_1755697866 for user test_user_123 has been deleted successfully"
}

=== Chat List After Deletion ===
{
  "chats": [
    {
      "chat_id": "chat_1755697807",
      "title": "Simple Math Question"
    }
  ]
}


## Performance Analysis

Analyze the performance and model routing decisions.

In [22]:
# Test different types of questions and track model usage
test_questions = [
    ("Simple", "What's the weather like?"),
    ("Medium", "Explain the concept of neural networks"),
    ("Hard", "Prove the convergence of the gradient descent algorithm for convex functions"),
    ("Code", "Write a Python implementation of quicksort algorithm"),
    ("Search", "Find the latest information about quantum computing breakthroughs")
]

model_usage = {}
performance_results = []

for category, question in test_questions:
    print(f"\n=== Testing {category} Question ===")
    chat_id = f"perf_test_{category.lower()}"
    
    start_time = time.time()
    response = send_message(TEST_USER_ID, chat_id, question)
    end_time = time.time()
    
    duration = end_time - start_time
    model = response.get('model', 'unknown')
    
    model_usage[model] = model_usage.get(model, 0) + 1
    performance_results.append({
        'category': category,
        'model': model,
        'duration': round(duration, 2),
        'has_citation': bool(response.get('citation'))
    })
    
    print(f"Model: {model}, Duration: {duration:.2f}s")
    if response.get('citation'):
        print("✅ Citations included")

print("\n=== Performance Summary ===")
print(f"Model Usage: {model_usage}")
print("\nDetailed Results:")
for result in performance_results:
    print(f"  {result['category']}: {result['model']} ({result['duration']}s) {'📚' if result['has_citation'] else ''}")


=== Testing Simple Question ===
Model: gemini-2.5-flash-lite, Duration: 3.06s

=== Testing Medium Question ===
Model: gemini-2.5-flash-lite, Duration: 3.06s

=== Testing Medium Question ===
Model: gemini-2.5-flash-lite, Duration: 7.37s

=== Testing Hard Question ===
Model: gemini-2.5-flash-lite, Duration: 7.37s

=== Testing Hard Question ===
Model: gemini-2.5-flash-lite, Duration: 8.41s

=== Testing Code Question ===
Model: gemini-2.5-flash-lite, Duration: 8.41s

=== Testing Code Question ===
Model: gemini-2.5-flash-lite, Duration: 10.52s

=== Testing Search Question ===
Model: gemini-2.5-flash-lite, Duration: 10.52s

=== Testing Search Question ===
Model: gemini-2.5-flash-lite, Duration: 11.20s
✅ Citations included

=== Performance Summary ===
Model Usage: {'gemini-2.5-flash-lite': 5}

Detailed Results:
  Simple: gemini-2.5-flash-lite (3.06s) 
  Medium: gemini-2.5-flash-lite (7.37s) 
  Hard: gemini-2.5-flash-lite (8.41s) 
  Code: gemini-2.5-flash-lite (10.52s) 
  Search: gemini-2.5-f

## Cleanup

Clean up test data (optional).

In [23]:
# Get all chats and delete test chats
print("=== Cleanup Test Chats ===")
all_chats = get_all_chats(TEST_USER_ID)

if 'chats' in all_chats:
    for chat in all_chats['chats']:
        chat_id = chat['chat_id']
        if chat_id.startswith(('chat_', 'perf_test_')):
            delete_response = delete_chat(TEST_USER_ID, chat_id)
            print(f"Deleted {chat_id}: {delete_response.get('message', 'Done')}")

print("\n=== Final Chat List ===")
final_chats = get_all_chats(TEST_USER_ID)
print(json.dumps(final_chats, indent=2))

=== Cleanup Test Chats ===
Deleted chat_1755697807: Chat chat_1755697807 for user test_user_123 has been deleted successfully
Deleted perf_test_simple: Chat perf_test_simple for user test_user_123 has been deleted successfully
Deleted perf_test_medium: Chat perf_test_medium for user test_user_123 has been deleted successfully
Deleted perf_test_hard: Chat perf_test_hard for user test_user_123 has been deleted successfully
Deleted perf_test_code: Chat perf_test_code for user test_user_123 has been deleted successfully
Deleted perf_test_search: Chat perf_test_search for user test_user_123 has been deleted successfully

=== Final Chat List ===
{
  "chats": []
}


## Conclusion

This notebook demonstrates all the key features of the FastAPI Gemini chatbot:

1. **Model Routing**: Automatically selects appropriate Gemini models based on content complexity
2. **Code Execution**: Handles code generation and execution with proper formatting
3. **Google Search Grounding**: Provides current information with citations
4. **Conversation Management**: Maintains chat history and generates titles
5. **CRUD Operations**: Create, read, and delete chat conversations
6. **Timeout Protection**: 25-second timeout to prevent hanging requests
7. **Error Handling**: Graceful handling of various error scenarios

The chatbot is production-ready with robust features for real-world usage.